# Sentiment analysis of show reviews

The goal of this analysis is to ...
* Get hands-on experience with packages and tools for analysing Russian language (natasha, nltk, spacy, rnnmorph, pymorphy2)
* Investigate available pre-trained models for Russian language (wor2vec, fasttext, navec, models from sber, deeppavlov and others)
* Learn how to finetune BERT-like models

## Imports

In [111]:
import gc
import os
import re
import sys
import warnings
from typing import List, Tuple

import dateparser
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

tqdm.pandas()

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [7]:
SEED = 42

## Data

### Loading data

In [27]:
%%time
reviews_df = pd.read_parquet("../../data/mt_reviews.parquet")
reviews_df.shape

CPU times: total: 3.92 s
Wall time: 3.93 s


(206737, 9)

### Dataset overview

In [28]:
reviews_df.sample(n=10, random_state=SEED)

,show_id,user_id,type,datetime,sentiment,subtitle,review_body,usefulness,scores
196236,257386,28525,movie,2010-08-27 11:23:00,good,Вечный город.,К истории Древнего мира у меня отношение особо...,40,
128582,688832,44953,movie,2015-02-20 23:19:00,bad,50 оттенков разочарования,"Говорю сразу, книги читала все, да и по нескол...",5,3 из 10
159673,349,33910,movie,2018-04-15 21:04:00,good,"Господи, спасибо, что не пронесло мимо","Есть два типа фильмов, мой друг. Одни ты прост...",4,
109244,686898,44065,movie,2019-11-09 11:58:00,neutral,Что же стало с клоуном?,"Итак, в первую очередь хотелось бы отметить то...",2,"6,5 из 10"
92610,61455,66782,movie,2017-11-27 18:52:00,good,Они отказываются подчиняться,"Автора этого замечательного фильма, Джосса Уэд...",5,
43727,491724,44563,movie,2012-01-28 23:18:00,good,Жестокая правда,"Финчер снова нас поразил, он всегда нас поража...",7,10 из 10
50195,102130,67145,movie,2009-08-17 11:56:00,good,"Преодолеть 2 года жизни, что встретиться","Слышал о фильме много, и в основном положитель...",11,10 из 10
43651,491724,66283,movie,2012-02-19 15:33:00,bad,"Мужчины, которые ненавидели женщин.",Я попробовала рассматривать этот фильм с двух ...,3,6 из 10
48775,7226,7905,movie,2014-02-20 03:44:00,good,None,"«Догвилль» - это один из тех редких фильмов, п...",3,10 из 10
75262,458,33255,movie,2013-06-22 21:14:00,good,Тайна закрытой двери,Я имела счастье смотреть этот мультфильм в кин...,6,10 из 10


### Looking at reviews

In [36]:
for review in reviews_df["review_body"].values[:10]:
    print(review.replace("<p>", "\n"))
    print("\n")

Варкрафт снял отличный режиссёр Данкан Джонс, который до этого создал великолепные Луна 2112 и Исходный код.
У данного кино по мотивам популярнейшей серии одноименных компьютерных игр непростая судьба. На зарубежных ресурсах фильм имеет в основном негативные отзывы, среди российских зрителей неожиданно позитивные.
Сюжет рассказывает как некая орда с очевидно ближневосточных по нашему восприятию земель, где закончились ресурсы и стало невыносимо жить, массово пытается переселиться к более цивилизованным людям. Люди этому активно сопротивляются. Никто поначалу даже не задумывается о межкультурном диалоге. Проблемы пытаются решить силой. Меж тем вопреки всему отдельные представили разных народов сумели найти общий язык и вместе решили избавиться от источника всех бед - скверны, которая поражает и развращает по обе стороны охранителей-пропагандистов - магов, наделённых властью. Именно с помощью скверны высасываются силы из простых и слабых существ.
По ходу истории, кстати, выясняется, что 

After looking at reviews we can see that many of them contain the manual scores like `7 из 10`, `5 из 10` and so on.  
Such scores will influence any model, especially models based on counts.  
It will be good if we extract such information and delete it from the reviews. 

### Parsing scores

In [113]:
def has_at_least_2_numbers(s: str) -> Tuple[bool, List[str]]:
    numbers = re.findall(r"\d(?:[,\d]*\.\d+|[,\d]*)", s)
    flag = len(numbers) > 1
    return flag, numbers

In [114]:
def convert_score(numbers: List[str]) -> float:
    numerator = float(numbers[-2])
    denominator = float(numbers[-1])
    return numerator / denominator if denominator > 0 else 0.0

In [115]:
def return_score(review: str) -> Tuple[float, str]:
    scores = []
    score = -1
    score_candidate = "N/A"
    for s in review.split("<p>")[-3:]:
        flag, numbers = has_at_least_2_numbers(s)
        if flag and (("/" in s) or ("из" in s)):
            return convert_score(numbers), s
        else:
            continue
    
    return -1, s

In [120]:
s = "Первым следует сказать что возрастной рейтинг 12+ или PG-13,использован в полную силу. И немного крови там присутствует. Ну а сам фильм Варкрафт в плане графики средний. Орки выглядят не так живо как я думал, зато магия сделана замечательно. А портал из мира Орков тоже неплох. Боевые сцены поставлены тоже неплохо. Увы но в сюжете есть некоторые дыры и несходные факты. Хотя конец фильма уж совсем не легко предугадать. Во время некоторых сцен я был весь на эмоциях. А также были затянутые сцены и сами диалоги в сценах были затянуты. Актёры стараются но выходит это нехорошо и неплохо, что-то среднее получается. Музыка в фильме то подходит атмосферой  к сцене то нет. Также драма здесь неплохая. Перейдём к главному, а именно к злодеям и героям. Злодей здесь один, и честно говоря его мотив это одно большое клише всех современных блокбастеров, он хочет захватить мир и потом им править. Главные герой интересные персонажи за которыми интересно наблюдать, поступки героев впринципе тяжело предугадать, а ещё больше всего из героев мне  понравился недо конца до ученый маг Кхадгар. В целом фильм вышел неплохим новым началом сказочной саги."

In [121]:
re.findall(r"\d(?:[,\d]*\.\d+|[,\d]*)", s)

['12', '13,']

In [119]:
reviews_df["review_body"][70]

'Первым следует сказать что возрастной рейтинг 12+ или PG-13,использован в полную силу. И немного крови там присутствует. Ну а сам фильм Варкрафт в плане графики средний. Орки выглядят не так живо как я думал, зато магия сделана замечательно. А портал из мира Орков тоже неплох. Боевые сцены поставлены тоже неплохо. Увы но в сюжете есть некоторые дыры и несходные факты. Хотя конец фильма уж совсем не легко предугадать. Во время некоторых сцен я был весь на эмоциях. А также были затянутые сцены и сами диалоги в сценах были затянуты. Актёры стараются но выходит это нехорошо и неплохо, что-то среднее получается. Музыка в фильме то подходит атмосферой  к сцене то нет. Также драма здесь неплохая. Перейдём к главному, а именно к злодеям и героям. Злодей здесь один, и честно говоря его мотив это одно большое клише всех современных блокбастеров, он хочет захватить мир и потом им править. Главные герой интересные персонажи за которыми интересно наблюдать, поступки героев впринципе тяжело предуга

In [116]:
scores, score_candidates = [], []

In [118]:
for i, review in tqdm(enumerate(reviews_df["review_body"].values)):
    print(i)
    score, score_candidate = return_score(review)
    scores.append(score)
    score_candidates.append(score_candidate)

70it [00:00, 35098.78it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70


ValueError: could not convert string to float: '13,'

In [97]:
score_candidates[:10]

['N/A',
 'По итогам, я могу сказать, что Варкрафт - это лучшая экранизация компьютерной игры, которую мы имеем на 2021 год. С большой вероятностью она понравится фанатам игры и любителям фентези-жанра.',
 '7 из 10',
 'N/A',
 '8 из 10',
 'N/A',
 'N/A',
 'N/A',
 '2 из 10',
 '5 из 10']

In [107]:
s = "По итогам, я могу сказать, что Варкрафт - это лучшая экранизация компьютерной игры, которую мы имеем на 2021 год. С большой вероятностью она понравится фанатам игры и любителям фентези-жанра."

In [108]:
re.findall(r"\d(?:[,\d]*\.\d+|[,\d]*)", s)

['2021']

### Selecting needed columns

For baseline model we're interested only in `sentiment` and `review_body` columns

In [10]:
df = reviews_df[["sentiment", "review_body"]]

In [11]:
del reviews_df
gc.collect()

441

### Splitting the data

In [12]:
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["sentiment"]
)
train_df.shape, test_df.shape

((165389, 2), (41348, 2))

In [13]:
train_df["sentiment"].value_counts(normalize=True)

good       0.720332
neutral    0.149974
bad        0.129694
Name: sentiment, dtype: float64

In [15]:
test_df["sentiment"].value_counts(normalize=True)

good       0.720325
neutral    0.149971
bad        0.129704
Name: sentiment, dtype: float64

## Modelling

### Hyperparameter Investigation

#### `lowercase`

In [16]:
%%time
# withour lowercase
vectorizer = CountVectorizer(lowercase=False)
vectors_wo_lowercase = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors_wo_lowercase.shape} with lowercase turned off"
)

The size of the train dataset is (165389, 753898) with lowercase turned off
CPU times: total: 35.5 s
Wall time: 35.5 s


In [17]:
%%time
# lowercase
vectorizer = CountVectorizer()
vectors_w_lowercase = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors_w_lowercase.shape} with lowercase turned on"
)

The size of the train dataset is (165389, 642874) with lowercase turned on
CPU times: total: 38.2 s
Wall time: 38.2 s


In [18]:
vectors_wo_lowercase.shape[1] - vectors_w_lowercase.shape[1]

111024

The difference in vocabulary size without making all characters lowercase and with lowercase is more than 100 000, so we better stick to lowercase 

#### `max_df` and `min_df`

In [19]:
vectorizer.get_feature_names_out()[:50]

array(['00', '000', '0000', '00000', '000000',
       '000000000000000000попкорн000000000000', '000000000000001',
       '000000000000на', '00000000000во', '00000000000данной',
       '00000000000есть000000000000000',
       '00000000000есть000000000000000000', '0000000000жевать',
       '0000000000ненавижу00000000', '00000000016', '000000000надо',
       '000000000разговаривать0000000000', '000000001',
       '00000000визуальная', '00000001', '000001', '00000громко',
       '00000точек', '00001', '0001', '0002', '000доктора', '000какой',
       '000км', '000косметические', '000теряются', '001', '002', '003',
       '00381', '006', '007', '00в', '00вых', '00е', '00м', '00по', '00с',
       '00седьмого', '00х', '00ые', '00ых', '01', '0100', '011'],
      dtype=object)

We can see that if we do not limit the vocabulary, we will have very infrequent words, so we better do it.  
For that we have to choose the `min_df` and `max_df` thresholds.

In [20]:
%%time
# min_df
vectorizer = CountVectorizer(min_df=0.8)
vectors = vectorizer.fit_transform(train_df["review_body"])
vectors.shape

CPU times: total: 34.7 s
Wall time: 34.7 s


(165389, 8)

In [21]:
vectorizer.get_feature_names_out()

array(['из', 'как', 'на', 'не', 'но', 'то', 'что', 'это'], dtype=object)

These words are in the 80% of all reviews and it is understandable.  
`из` is there because many reviews contain a score like `7.6 из 10` and other words are just common.  

In [22]:
%%time
# min_df
MIN_DF = 0.01
vectorizer = CountVectorizer(min_df=MIN_DF)
vectors = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors.shape} with lowercase turned on and min_df={MIN_DF}"
)

The size of the train dataset is (165389, 3352) with lowercase turned on and min_df=0.01
CPU times: total: 35.1 s
Wall time: 35.1 s


In [23]:
vectorizer.get_feature_names_out()[:50]

array(['10', '100', '11', '12', '13', '15', '16', '17', '18', '20',
       '2012', '21', '30', '3d', '40', '50', '60', '70', '80', '90', 'dc',
       'marvel', 'of', 'the', 'абсолютно', 'аватар', 'автор', 'автора',
       'авторов', 'авторы', 'агент', 'аж', 'актер', 'актера', 'актерам',
       'актерами', 'актерах', 'актеров', 'актером', 'актерская',
       'актерский', 'актерского', 'актерской', 'актерскую', 'актеры',
       'актриса', 'актрисы', 'актёр', 'актёра', 'актёров'], dtype=object)

In [69]:
%%time
# ngram_range
NGRAM_RANGE = (1, 3)
vectorizer = CountVectorizer(ngram_range=NGRAM_RANGE, min_df=MIN_DF)
train_vectors = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors.shape} with lowercase turned on and min_df={MIN_DF} and ngram_range={NGRAM_RANGE}"
)

The size of the train dataset is (165389, 4840) with lowercase turned on and min_df=0.01 and ngram_range=(1, 3)
CPU times: total: 4min 28s
Wall time: 10min 16s


In [70]:
vectorizer.get_feature_names_out()[:50]

array(['10', '10 за', '10 из', '10 из 10', '10 лет', '100', '11', '12',
       '13', '15', '16', '17', '18', '20', '2012', '21', '30', '3d', '40',
       '50', '60', '70', '80', '90', 'dc', 'marvel', 'of', 'the',
       'абсолютно', 'абсолютно все', 'абсолютно не', 'аватар', 'автор',
       'автора', 'авторов', 'авторы', 'агент', 'аж', 'актер', 'актера',
       'актерам', 'актерами', 'актерах', 'актеров', 'актером',
       'актерская', 'актерская игра', 'актерский', 'актерский состав',
       'актерского'], dtype=object)

In [71]:
test_vectors = vectorizer.transform(test_df["review_body"])

### Label Encoding

In [73]:
le = LabelEncoder()
train_labels = le.fit_transform(train_df["sentiment"])
test_labels = le.transform(test_df["sentiment"])

In [ ]:
%%time
svc = svm.SVC()
svc.fit(train_vectors, train_labels)

In [ ]:
f1_score(test_labels, svc.predict(test_vectors))

### Tf-Idf for `review_body`

In [24]:
vectorizer_params = {
    "ngram_range": (1, 2),
    "max_features": 10000,
    "tokenizer": lambda s: s.split(),
}
vectorizer_article = TfidfVectorizer(**vectorizer_params)

In [25]:
%%timeit
X_train_review = vectorizer_article.fit_transform(train_df["review_body"])


KeyboardInterrupt



In [ ]:
%%timeit
X_test_review = vectorizer_article.transform(test_df["review_body"])

### Training LogReg

In [20]:
log_reg = LogisticRegression(random_state=SEED)